In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
from scipy import ndimage
import os

Mounted at /content/drive


In [ ]:
''' 
Below is the function to normalize the pixels between 0 and 1 
'''

def normalize(image):
    image[image<0] = 0. # bring all the negative values to 0, otherwise the normalization doesn't work since they remain negative also after the normalization
    image = (image - image.min()) / (image.max() -image.min())

    return image

'''
Below are the functions to enhance the image (sharpening the image, increasing the contrast) and to reduce the noise
'''

def enhancement (image):
  filtered = ndimage.gaussian_filter(image,0.2)
  alpha = 30
  sharpened = image + alpha * (image - filtered)

  minval = np.percentile(sharpened, 30)
  maxval = np.percentile(sharpened, 99.9)
  increase_contrast = np.clip(sharpened, minval, maxval)
  increase_contrast = ((increase_contrast - minval) / (maxval - minval)) * 255
  
  enhanced = increase_contrast
  return enhanced


def reduce_noise(image, n = 6):
    reduce_noise = np.zeros((n, n) + (image.shape[0] - n + 1, image.shape[1] - n + 1) + image.shape[2:], dtype = image.dtype)
    for i in range(n):
        for j in range(n):
            reduce_noise[i, j] = image[i:i + reduce_noise.shape[2], j:j + reduce_noise.shape[3]]
    reduce_noise = np.moveaxis(reduce_noise, (0, 1), (-2, -1)).reshape(*reduce_noise.shape[2:], -1)
    reduce_noise = np.median(reduce_noise, axis = -1)
    if reduce_noise.dtype != image.dtype:
        reduce_noise = (reduce_noise.astype(np.float64) + 10 ** -7).astype(image.dtype)
    return reduce_noise

In [ ]:
''' INSERTING ALL THE SCANS IN A VECTOR '''

scans = []
path = '/content/drive/MyDrive/...' # add path of preprocessed scans
for r, d, f in os.walk(path):
  for file in f: 
    if file.endswith('.npy'):
      scans.append(os.path.join(r, file))

In [ ]:
i = 0
for i in range(len(scans)):
  scan = np.load(scans[i])
  enhanced = np.zeros(scan.shape, dtype = int)
  reduced_noise = np.zeros([scan.shape[0], scan.shape[1]-5, scan.shape[2]-5] , dtype = int) #this array needs a different dimension, because the function reduce_noise reduces the dimension of both x and y axes of 5
  for j in range(scan.shape[0]):
    enhanced[j] = enhancement(scan[j])
    reduced_noise[j] = reduce_noise(enhanced[j])
  reduced_noise = reduced_noise.astype(np.int16)
  np.save('/content/drive/MyDrive/.../Enhanced-scan-'+str(i+1), reduced_noise) #insert path where to save the enhanced scan